In [1]:
from google.colab import drive
from os import chdir, getcwd

drive.mount('/content/gdrive')
chdir("/content/gdrive/My Drive/SSD_ADAS/ssd_keras/")
print("Working directory:", getcwd())

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Working directory: /content/gdrive/My Drive/SSD_ADAS/ssd_keras


In [2]:
from data_generator.object_detection_2d_data_generator import DataGenerator

input_format = ["image_name", "class_id", "xmin", "xmax", "ymin", "ymax"] # order of columns in the annotation csv

# Load validation data
validation_image_dir = "../FLIR_ADAS/validation/PreviewData"
validation_annotation_csv = "../validation.csv"

validation_dataset = DataGenerator()
validation_dataset.parse_csv(
    validation_image_dir,
    validation_annotation_csv,
    input_format
)
print("{} images in validation set.".format(validation_dataset.get_dataset_size()))

# Load training data
training_image_dir = "../FLIR_ADAS/training/PreviewData"
training_annotation_csv = "../training.csv"

training_dataset = DataGenerator()
training_dataset.parse_csv(
    training_image_dir,
    training_annotation_csv,
    input_format
)
print("{} images in training set.".format(training_dataset.get_dataset_size()))

1360 images in validation set.
7860 images in training set.


In [3]:
from keras import backend as K
from keras.models import load_model

from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_L2Normalization import L2Normalization
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_loss_function.keras_ssd_loss import SSDLoss

def load_ssd(h5_filename):
  #Clear previously loaded models
  K.clear_session()
  
  #Fill in the custom objects so that Keras can load the model
  ssd_loss = SSDLoss(neg_pos_ratio=3, alpha=1.0)
  
  custom_objects = {
      'AnchorBoxes': AnchorBoxes,
      'L2Normalization': L2Normalization,
      'DecodeDetections': DecodeDetections,
      'compute_loss': ssd_loss.compute_loss}
  
  #Load the model
  model = load_model(h5_filename, custom_objects)
  print("Model loaded:", h5_filename)
  
  return model
  
model = load_ssd("ssd300_untrained_coco.h5")

Using TensorFlow backend.


Model loaded: ssd300_untrained_coco.h5


In [0]:
from ssd_encoder_decoder.ssd_input_encoder import SSDInputEncoder

img_height = 300
img_width = 300
n_classes = 80 # COCO classes

scales = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05]
aspect_ratios_per_layer=[[1.0, 2.0, 0.5],
                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                         [1.0, 2.0, 0.5, 3.0, 1.0/3.0],
                         [1.0, 2.0, 0.5],
                         [1.0, 2.0, 0.5]]

two_boxes_for_ar1=True
steps=[8, 16, 32, 64, 100, 300]
offsets=[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
clip_boxes=False
variances=[0.1, 0.1, 0.2, 0.2]
normalize_coords=True

# The encoder constructor needs the spatial dimensions of the model's predictor layers to create the anchor boxes.
predictor_sizes = [model.get_layer('conv4_3_norm_mbox_conf').output_shape[1:3],
                   model.get_layer('fc7_mbox_conf').output_shape[1:3],
                   model.get_layer('conv6_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv7_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv8_2_mbox_conf').output_shape[1:3],
                   model.get_layer('conv9_2_mbox_conf').output_shape[1:3]]

ssd_input_encoder = SSDInputEncoder(img_height=img_height,
                                    img_width=img_width,
                                    n_classes=n_classes,
                                    predictor_sizes=predictor_sizes,
                                    scales=scales,
                                    aspect_ratios_per_layer=aspect_ratios_per_layer,
                                    two_boxes_for_ar1=two_boxes_for_ar1,
                                    steps=steps,
                                    offsets=offsets,
                                    clip_boxes=clip_boxes,
                                    variances=variances,
                                    matching_type='multi',
                                    pos_iou_threshold=0.5,
                                    neg_iou_limit=0.5,
                                    normalize_coords=normalize_coords)

In [0]:
from data_generator.object_detection_2d_geometric_ops import Resize, ResizeRandomInterp
from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
from data_generator.data_augmentation_chain_original_ssd import SSDExpand, SSDRandomCrop
from data_generator.object_detection_2d_geometric_ops import RandomFlip
from data_generator.object_detection_2d_image_boxes_validation_utils import BoxFilter

import cv2

mean_color = [128, 128, 128]

# For validation images
convert_to_3_channels = ConvertTo3Channels()
resize = Resize(height=img_height, width=img_width)

# SSD augmentation chain without photometric ops
expand = SSDExpand(background=mean_color)
random_crop = SSDRandomCrop()
random_flip = RandomFlip(dim='horizontal', prob=0.5)
box_filter = BoxFilter(check_overlap=False, check_min_area=False, check_degenerate=True)
resize_random_interp = ResizeRandomInterp(
    height=img_height,
    width=img_width,
    interpolation_modes=[cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_CUBIC, cv2.INTER_AREA, cv2.INTER_LANCZOS4],
    box_filter=box_filter)

In [0]:
# Create training and validation generators
batch_size = 8

train_generator = training_dataset.generate(
    batch_size=batch_size,
    shuffle=True,
    transformations=[convert_to_3_channels, expand, random_crop, random_flip, resize_random_interp],
    label_encoder=ssd_input_encoder,
    returns={'processed_images', 'encoded_labels'},
    keep_images_without_gt=False)

val_generator = validation_dataset.generate(
    batch_size=batch_size,
    shuffle=False,
    transformations=[convert_to_3_channels, resize],
    label_encoder=ssd_input_encoder,
    returns={'processed_images', 'encoded_labels'},
    keep_images_without_gt=False)

In [0]:
# Define model callbacks.

from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TerminateOnNaN

model_checkpoint = ModelCheckpoint(
    filepath='ssd300_geometric_aug_only_epoch-{epoch:02d}_loss-{loss:.4f}_val_loss-{val_loss:.4f}.h5',
    monitor='val_loss',
    verbose=1,
    save_best_only=True)

csv_logger = CSVLogger(
    filename='ssd300_ADAS_geometric_aug_only_training_log.csv',
    append=True)

reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    verbose=1,
    min_delta=0.0001)

terminate_on_nan = TerminateOnNaN()

callbacks = [model_checkpoint,
             csv_logger,
             reduce_lr_on_plateau,
             terminate_on_nan]

In [0]:
# Train the model

from math import ceil

initial_epoch   = 0
final_epoch     = 120
steps_per_epoch = 1000

history = model.fit_generator(
    generator=train_generator,
    steps_per_epoch=steps_per_epoch,
    epochs=final_epoch,
    callbacks=callbacks,
    validation_data=val_generator,
    validation_steps=ceil(validation_dataset.get_dataset_size()/batch_size),
    initial_epoch=initial_epoch)

Epoch 1/120
1000/1000 [==============================] - 2288s 2s/step - loss: 7.3432 - val_loss: 5.6286

Epoch 00001: val_loss improved from inf to 5.62859, saving model to ssd300_geometric_aug_only_epoch-01_loss-7.3441_val_loss-5.6286.h5
Epoch 2/120
1000/1000 [==============================] - 854s 854ms/step - loss: 5.9997 - val_loss: 6.0892

Epoch 00002: val_loss did not improve from 5.62859
Epoch 3/120
1000/1000 [==============================] - 853s 853ms/step - loss: 5.5768 - val_loss: 4.8706

Epoch 00003: val_loss improved from 5.62859 to 4.87060, saving model to ssd300_geometric_aug_only_epoch-03_loss-5.5767_val_loss-4.8706.h5
Epoch 4/120
1000/1000 [==============================] - 853s 853ms/step - loss: 5.3101 - val_loss: 4.7694

Epoch 00004: val_loss improved from 4.87060 to 4.76940, saving model to ssd300_geometric_aug_only_epoch-04_loss-5.3099_val_loss-4.7694.h5
Epoch 5/120
1000/1000 [==============================] - 851s 851ms/step - loss: 5.1199 - val_loss: 4.4194

E